# COGS 108 - Data Checkpoint

# Names

- Fadi Elhayek
- Ivy Duggan (lead on data checkpoint)
- Joshua Guiman
- Yu-Hsuan Chi

<a id='research_question'></a>
# Research Question

Are there any key indicators on a game's store page as to whether or not it is good? (ie. Can this be determined through sentiment analysis of the name/blurb/genres, the mismatch of developer & user tagged genres, recent/overall reviews, release date, metacritic score, and maturity rating, etc.?)

# Dataset(s)


(Copy this information for each dataset)
- Dataset Name: df_clean
- Link to the dataset: contained in this notebook in `df_clean.csv`
- Number of observations: ~75,000 rows * ~13 columns

This is data based on games from store.steampowered.com, after being scraped by my custom code below.
It follows this format
- `'app_id'`: unique numeric id
- `'game_name'`: name of game
- `'release_date'`: release date in datetime format
- `'genre'`: string genre
- `'game_blurb'`: string blurb
- `'maturity_rating_score'`: string score from any agency
- `'maturity_rating_agency'`: rating agency
- `'esrb_rating_numeric'`: esrb letter rating mapped into number
- `'total_review_score_numeric'`: total reviews mapped into numbers
- `'recent_review_score_numeric'`: recent reviews mapped into numbers
- `'review_count'`: total number of reviews
- `'critic_score'`: metacritic score, if there is one
- `'critic_name'`: name of scoring site, currently only using metacritic
- `'user_tags_list'`: list of tags that users can add to games
- `'developer_genres_list'`: genres that the developer/publisher have tagged the game under
- `'game_features_list'`: list of steam specific features the developer/publisher supports

# Setup

store.steampowered.com does not publicly list all app ids, so we first have to scrape for them. We use the base URL `https://store.steampowered.com/search/?category1=998&filter=topsellers&ignore_preferences=1&page=1`

This has the following important parts:
- `category1=998`: this is the category for games, not software/music/etc.
- `ignore_preferences=1`: this removes all personalization in ordering of games
- `page=1`: the website uses infinite scroll, which is hard to parse. we force this pagination since it was never removed

We scrape this results page for links in the following format:
`https://store.steampowered.com/app/1085660/Destiny_2/?snr=1_7_7_7000_150_1` where `1085660` is the app id that we need a list of to have valid games. App IDs are not continuous (known range is from 2-7 digits)

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import requests
import threading

def extract_game_links(soup):
    '''extract the <a> tags that have the correct class to be links to games'''
    try:
        # search_resultsRows is the element containing the list of games
        results = soup.find(id='search_resultsRows')

        # search_result_row is the element that should have our game link/app id itself
        return results.find_all('a', class_='search_result_row')
    except:
        return None

def extract_app_id(link):
    '''take a link and remove the app id if it is an app'''
    try:
        link_parts = link['href'].split('/')
        
        # double check to make sure we only deal with full apps and not software, etc.
        if link_parts[3] == 'app':
            return link_parts[4] # this is numeric app id
        else:
            return None
    except:
        return None

def get_game_links(start_page, end_page):
    '''scrape multiple pages for links and return all links at the end'''
    game_links = []
    for i in range(start_page, end_page):
        if i % 50 == 0: print('on page ' + str(i))
        URL = f'https://store.steampowered.com/search/?category1=998&ignore_preferences=1&page={i}'
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, 'html.parser')
        game_links = game_links + extract_game_links(soup)
        
    return game_links

Steam has ~75,000 games on its site, which at ~25 per page, is 3000 pages maximum. This takes a while to run, so the following loop has been made for this part. We then store these valid app_ids so we never have to redo this step, hopefully. Broken up into multiple parts since it may crash the kernel

In [2]:
def get_valid_app_ids(start, end):
    game_links = get_game_links(start, end) # NOTE: DO NOT UNCOMMENT IF YOU ARE NOT READY THIS TAKES A WHILE

    valid_app_ids = []
    for i in range(len(game_links)):
        if i % 100 == 0: print('completed ' + str(i))
        valid_app_ids.append(extract_app_id(game_links[i]))

    return valid_app_ids

In [3]:
block_size = 500 # 500 is max for 3000 pages total
valid_app_ids_1 = get_valid_app_ids(0, block_size)

on page 0
on page 50
on page 100
on page 150
on page 200
on page 250
on page 300
on page 350
on page 400
on page 450
completed 0
completed 100
completed 200
completed 300
completed 400
completed 500
completed 600
completed 700
completed 800
completed 900
completed 1000
completed 1100
completed 1200
completed 1300
completed 1400
completed 1500
completed 1600
completed 1700
completed 1800
completed 1900
completed 2000
completed 2100
completed 2200
completed 2300
completed 2400
completed 2500
completed 2600
completed 2700
completed 2800
completed 2900
completed 3000
completed 3100
completed 3200
completed 3300
completed 3400
completed 3500
completed 3600
completed 3700
completed 3800
completed 3900
completed 4000
completed 4100
completed 4200
completed 4300
completed 4400
completed 4500
completed 4600
completed 4700
completed 4800
completed 4900
completed 5000
completed 5100
completed 5200
completed 5300
completed 5400
completed 5500
completed 5600
completed 5700
completed 5800
completed 

In [5]:
valid_app_ids_2 = get_valid_app_ids(block_size + 1, block_size * 2)

on page 550
on page 600
on page 650
on page 700
on page 750
on page 800
on page 850
on page 900
on page 950
completed 0
completed 100
completed 200
completed 300
completed 400
completed 500
completed 600
completed 700
completed 800
completed 900
completed 1000
completed 1100
completed 1200
completed 1300
completed 1400
completed 1500
completed 1600
completed 1700
completed 1800
completed 1900
completed 2000
completed 2100
completed 2200
completed 2300
completed 2400
completed 2500
completed 2600
completed 2700
completed 2800
completed 2900
completed 3000
completed 3100
completed 3200
completed 3300
completed 3400
completed 3500
completed 3600
completed 3700
completed 3800
completed 3900
completed 4000
completed 4100
completed 4200
completed 4300
completed 4400
completed 4500
completed 4600
completed 4700
completed 4800
completed 4900
completed 5000
completed 5100
completed 5200
completed 5300
completed 5400
completed 5500
completed 5600
completed 5700
completed 5800
completed 5900
comp

In [6]:
valid_app_ids_3 = get_valid_app_ids(block_size * 2 + 1, block_size * 3)

on page 1050
on page 1100
on page 1150
on page 1200
on page 1250
on page 1300
on page 1350
on page 1400
on page 1450
completed 0
completed 100
completed 200
completed 300
completed 400
completed 500
completed 600
completed 700
completed 800
completed 900
completed 1000
completed 1100
completed 1200
completed 1300
completed 1400
completed 1500
completed 1600
completed 1700
completed 1800
completed 1900
completed 2000
completed 2100
completed 2200
completed 2300
completed 2400
completed 2500
completed 2600
completed 2700
completed 2800
completed 2900
completed 3000
completed 3100
completed 3200
completed 3300
completed 3400
completed 3500
completed 3600
completed 3700
completed 3800
completed 3900
completed 4000
completed 4100
completed 4200
completed 4300
completed 4400
completed 4500
completed 4600
completed 4700
completed 4800
completed 4900
completed 5000
completed 5100
completed 5200
completed 5300
completed 5400
completed 5500
completed 5600
completed 5700
completed 5800
completed 

In [7]:
valid_app_ids_4 = get_valid_app_ids(block_size * 3 + 1, block_size * 4)

on page 1550
on page 1600
on page 1650
on page 1700
on page 1750
on page 1800
on page 1850
on page 1900
on page 1950
completed 0
completed 100
completed 200
completed 300
completed 400
completed 500
completed 600
completed 700
completed 800
completed 900
completed 1000
completed 1100
completed 1200
completed 1300
completed 1400
completed 1500
completed 1600
completed 1700
completed 1800
completed 1900
completed 2000
completed 2100
completed 2200
completed 2300
completed 2400
completed 2500
completed 2600
completed 2700
completed 2800
completed 2900
completed 3000
completed 3100
completed 3200
completed 3300
completed 3400
completed 3500
completed 3600
completed 3700
completed 3800
completed 3900
completed 4000
completed 4100
completed 4200
completed 4300
completed 4400
completed 4500
completed 4600
completed 4700
completed 4800
completed 4900
completed 5000
completed 5100
completed 5200
completed 5300
completed 5400
completed 5500
completed 5600
completed 5700
completed 5800
completed 

In [8]:
valid_app_ids_5 = get_valid_app_ids(block_size * 4 + 1, block_size * 5)

on page 2050
on page 2100
on page 2150
on page 2200
on page 2250
on page 2300
on page 2350
on page 2400
on page 2450
completed 0
completed 100
completed 200
completed 300
completed 400
completed 500
completed 600
completed 700
completed 800
completed 900
completed 1000
completed 1100
completed 1200
completed 1300
completed 1400
completed 1500
completed 1600
completed 1700
completed 1800
completed 1900
completed 2000
completed 2100
completed 2200
completed 2300
completed 2400
completed 2500
completed 2600
completed 2700
completed 2800
completed 2900
completed 3000
completed 3100
completed 3200
completed 3300
completed 3400
completed 3500
completed 3600
completed 3700
completed 3800
completed 3900
completed 4000
completed 4100
completed 4200
completed 4300
completed 4400
completed 4500
completed 4600
completed 4700
completed 4800
completed 4900
completed 5000
completed 5100
completed 5200
completed 5300
completed 5400
completed 5500
completed 5600
completed 5700
completed 5800
completed 

In [9]:
valid_app_ids_6 = get_valid_app_ids(block_size * 5 + 1, block_size * 6)

on page 2550
on page 2600
on page 2650
on page 2700
on page 2750
on page 2800
on page 2850
on page 2900
on page 2950
completed 0
completed 100
completed 200
completed 300
completed 400
completed 500
completed 600
completed 700
completed 800
completed 900
completed 1000
completed 1100
completed 1200
completed 1300
completed 1400
completed 1500
completed 1600
completed 1700
completed 1800
completed 1900
completed 2000
completed 2100
completed 2200
completed 2300
completed 2400
completed 2500
completed 2600
completed 2700
completed 2800
completed 2900
completed 3000
completed 3100
completed 3200
completed 3300
completed 3400
completed 3500
completed 3600
completed 3700
completed 3800
completed 3900
completed 4000
completed 4100
completed 4200
completed 4300
completed 4400
completed 4500
completed 4600
completed 4700
completed 4800
completed 4900
completed 5000
completed 5100
completed 5200
completed 5300
completed 5400
completed 5500
completed 5600
completed 5700
completed 5800
completed 

In [2]:
valid_app_ids = valid_app_ids_1 + valid_app_ids_2 + valid_app_ids_3 + valid_app_ids_4 + valid_app_ids_5 + valid_app_ids_6
# need to remove Nones from this array
print(len(valid_app_ids)) # sanity check!
valid_app_ids = [i for i in valid_app_ids if i is not None]

print(len(valid_app_ids)) # sanity check!

valid_app_ids = list(dict.fromkeys(valid_app_ids)) # dedupe
print(len(valid_app_ids)) # sanity check!

# save to a file for convenience
%store valid_app_ids >valid_app_ids

NameError: name 'valid_app_ids_1' is not defined

In [3]:
# Use this cell to grab the written valid app ids instead of using the code above

import re

valid_app_ids_file = open("valid_app_ids", "r")
with open('valid_app_ids') as f:
    valid_app_ids = f.read().splitlines()
    
for i in range(len(valid_app_ids)):
    valid_app_ids[i] = re.sub(r'\W+', '', valid_app_ids[i])
    
print(len(valid_app_ids))

66266


Next we need to actually get the data for every game. I've written all of these functions to scrape the store page for the relevant data. It is quite verbose, but pretty basic grabbing of specific text values.

In [4]:
def get_game_name(soup):
    '''extract game name'''
    try:
        name = soup.find(id='appHubAppName').text
    except AttributeError:
        return None
    return name

def get_release_date(soup):
    '''extract release date'''
    try:
        date_object = soup.find(class_="release_date")
        date = date_object.find(class_="date").text
    except AttributeError:
        return None
    return date

def get_game_blurb(soup):
    '''extract short game description'''
    try:
        game_blurb = str.strip(soup.find(class_='game_description_snippet').text)
    except AttributeError:
            return None
    return game_blurb

# genre at the top next to "All Games >"
# also includes franchises sometimes
def get_base_genre_and_franchise(soup, app_id):
    '''extract base genre and also franchises (franchises is unused so far)'''
    try:
        genre_container = soup.find(class_='breadcrumbs')
        links = genre_container.find_all("a")
        genres = []
        for link in links:
            link = link
            if link.text != 'All Games' and str(app_id) not in link['href']:
                genres.append(link.text)
    except AttributeError:
        return [None]    
    return genres

def get_user_tags(soup):
    '''grab all user voted tags'''
    try:
        # note: not all tags that we can get here are displayed at once on the page
        # when viewed by a person
        user_tags_object = soup.find_all(class_="app_tag")
        user_tags = [str.strip(item.text) for item in user_tags_object]
    except AttributeError:
        return [None]
    return user_tags

def get_maturity_rating(soup):
    '''grab the maturity rating of the game and also the specific rating agency'''
    try:
        rating_object = soup.find(class_='shared_game_rating')
        rating_link = rating_object.find('img')['src']
        rating = rating_link.split('/')[-1][0].upper()

        rating_agency_blurb = rating_object.find(class_='game_rating_agency').text
        rating_agency = rating_agency_blurb.split(' ')[-1]
        # unsure if there are multiple ratings
    except AttributeError:
        return None, None
    return rating, rating_agency

def get_game_features(soup):
    '''this extracts a list of game features steam advertises on game store pages'''
    try:
        game_features_object = soup.find(class_='game_area_features_list_ctn')

        game_features = [str.strip(item.text) for item in game_features_object.find_all(class_='label')]
    except AttributeError:
            return [None]

    return game_features

def get_developer_genres(soup):
    '''in addition to the base genre above, there can be multiple developer tagged genres for a game'''
    try:
        genres_object = soup.find(id='genresAndManufacturer')
        # specific selector for genres, not developers
        genres = [item.text for item in genres_object.find('span').find_all('a')]
    except AttributeError:
            return [None]
    return genres

def get_critic_score(soup):
    '''extract the critic score and which source (currently using metacritic only)'''
    try:
        score_object = soup.find(id='game_area_metascore')
        score = str.strip(score_object.find(class_='score').text)
        critic = score_object.find(class_='wordmark').find_all('div')[0].text
    except AttributeError:
            return None, None
    return score, critic

def get_reviews(soup):
    '''grab the recent review score, the total review score, and the number of reviews'''
    try:
        review_count_object = soup.find('input', type="hidden", id='review_summary_num_reviews')
        review_count = review_count_object['value'] if review_count_object is not None else -1
        reviews_object = soup.find(id='review_histograms_container')
        review_text_selector = lambda item: item.find(class_='game_review_summary').text

        total_reviews = reviews_object.find(id='review_histogram_rollup_section')
        recent_reviews = reviews_object.find(id='review_histogram_recent_section')

        total_review_text = review_text_selector(total_reviews)
        recent_review_text = review_text_selector(recent_reviews)
    except AttributeError:
        return None, None, None
    return total_review_text, recent_review_text, review_count

In [5]:
def create_game_data_entry(app_id, output = False, verbose = False):
    '''call all of the extract methods on a specific app id and return a row of data'''
    if output: print('Start: ' + app_id + ' +++++')
        
    app_url = f'https://store.steampowered.com/app/{app_id}'
    page = requests.get(app_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    game_name = get_game_name(soup)    
    entry = [
        app_id,
        game_name,
        get_release_date(soup),
        get_game_blurb(soup),
        get_developer_genres(soup),
        get_user_tags(soup),
        get_game_features(soup),
        get_maturity_rating(soup),
        get_base_genre_and_franchise(soup, app_id),
        get_critic_score(soup),
        get_reviews(soup),
    ]
    
    if output and verbose: print(entry)
    if output: print('End ' + app_id + ' -----')
    return entry

Now that we have a function that can create entries with all of our helper functions, we can create our column names and populate the dataframe

In [6]:
col_names = (
    'app_id',
    'game_name',
    'release_date',
    'game_blurb',
    'developer_genres_list',
    'user_tags_list',
    'game_features_list',
    'maturity_rating_data',
    'genre_and_franchise_data',
    'critic_data',
    'reviews_data',
    )
df = pd.DataFrame(columns=col_names)

class GameDataThread(threading.Thread):
    def __init__(self, app_ids):
        threading.Thread.__init__(self)
        self.app_ids = app_ids
        self.game_entries = []
 
    def run(self):
        '''create data entries for a subset of valid app ids'''
        log_counter = 0
        for app_id in self.app_ids:
            if log_counter % 500 == 0: 
                print(str(threading.current_thread().name) + ' on ' + str(log_counter))
            self.game_entries.append(create_game_data_entry(app_id))
            log_counter += 1

n_threads = 8
threads = []
block_size = int((len(valid_app_ids)/n_threads))

print('creating threads')
for i in range(n_threads):
    start_index = i * block_size
    end_index = (i + 1) * block_size + 1
    thread = GameDataThread(valid_app_ids[start_index:end_index])
    threads.append(thread)
    
print('starting threads')
for thread in threads:
    thread.start()
    
print('combining thread results')
for thread in threads:
    print('joining ' + thread.getName())
    thread.join()
    

creating threads
starting threads
Thread-5 on 0
Thread-6 on 0
Thread-7 on 0
Thread-8 on 0
Thread-9 on 0
Thread-10 on 0
Thread-11 on 0
Thread-12 on 0
combining thread results
joining Thread-5
Thread-5 on 500
Thread-12 on 500
Thread-9 on 500
Thread-11 on 500
Thread-6 on 500
Thread-7 on 500
Thread-10 on 500
Thread-8 on 500
Thread-5 on 1000
Thread-12 on 1000
Thread-9 on 1000
Thread-11 on 1000
Thread-6 on 1000
Thread-7 on 1000
Thread-8 on 1000
Thread-10 on 1000


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Thread-5 on 1500
Thread-12 on 1500
Thread-9 on 1500
Thread-11 on 1500
Thread-6 on 1500
Thread-7 on 1500
Thread-8 on 1500
Thread-10 on 1500
Thread-12 on 2000
Thread-5 on 2000
Thread-9 on 2000
Thread-11 on 2000
Thread-7 on 2000
Thread-6 on 2000
Thread-8 on 2000
Thread-10 on 2000
Thread-12 on 2500
Thread-5 on 2500
Thread-11 on 2500
Thread-9 on 2500
Thread-6 on 2500
Thread-7 on 2500
Thread-8 on 2500
Thread-10 on 2500
Thread-12 on 3000
Thread-5 on 3000
Thread-11 on 3000
Thread-9 on 3000
Thread-6 on 3000
Thread-7 on 3000
Thread-8 on 3000
Thread-10 on 3000
Thread-5 on 3500
Thread-12 on 3500
Thread-11 on 3500
Thread-9 on 3500
Thread-6 on 3500
Thread-8 on 3500
Thread-7 on 3500
Thread-10 on 3500
Thread-5 on 4000
Thread-12 on 4000
Thread-11 on 4000
Thread-9 on 4000
Thread-6 on 4000
Thread-8 on 4000
Thread-7 on 4000
Thread-10 on 4000
Thread-5 on 4500
Thread-12 on 4500
Thread-11 on 4500
Thread-9 on 4500
Thread-6 on 4500
Thread-8 on 4500
Thread-7 on 4500
Thread-10 on 4500
Thread-5 on 5000
Thread-12 

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Thread-11 on 6000
Thread-9 on 6000
Thread-12 on 6500
Thread-6 on 6000
Thread-5 on 6500
Thread-8 on 6000
Thread-7 on 6000
Thread-10 on 6000
Thread-11 on 6500
Thread-12 on 7000
Thread-9 on 6500
Thread-8 on 6500
Thread-7 on 6500
Thread-11 on 7000
Thread-5 on 7000
Thread-10 on 6500
Thread-6 on 6500
Thread-12 on 7500
Thread-11 on 7500
Thread-9 on 7000
Thread-8 on 7000
Thread-7 on 7000
Thread-10 on 7000
Thread-5 on 7500
Thread-6 on 7000
Thread-12 on 8000
Thread-11 on 8000
Thread-7 on 7500
Thread-8 on 7500
Thread-9 on 7500
Thread-10 on 7500
Thread-5 on 8000
Thread-6 on 7500
on entry #0
joining Thread-6
Thread-7 on 8000
Thread-8 on 8000
Thread-9 on 8000
Thread-10 on 8000
Thread-6 on 8000
on entry #0


C:\Users\ethan\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


joining Thread-7
on entry #0
joining Thread-8
on entry #0
joining Thread-9
on entry #0
joining Thread-10
on entry #0
joining Thread-11
on entry #0
joining Thread-12
on entry #0


,app_id,game_name,release_date,game_blurb,developer_genres_list,user_tags_list,game_features_list,maturity_rating_data,genre_and_franchise_data,critic_data,reviews_data
0,1768830,Frogman Magmaborn,2022,A story-based rogue-like startegy RPG. Discove...,"[Free to Play, Indie, RPG, Strategy]","[RPG, Strategy, Roguelike, Level Editor, Modda...","[Single-player, Includes level editor, Steam i...","(None, None)",[Strategy Games],"(None, None)","(None, None, None)"
1,1768920,Splatter,December 2022,Splatter is a first-person fever dream where y...,"[Action, Indie]","[Action, FPS, Shooter, 3D, Colorful, First-Per...","[Single-player, Steam Achievements, Captions a...","(None, None)",[Action Games],"(None, None)","(None, None, None)"
2,1770010,Airport Renovator: Prologue,Wishlist now to get notified!,The airport is devastated. It’s up to you to b...,"[Casual, Indie, Simulation]","[Life Sim, Simulation, Casual, FPS, Open World...","[Single-player, Steam is learning about this g...","(None, None)",[Simulation Games],"(None, None)","(None, None, None)"
3,1786150,Horror Drift (ホラードリフト),2022,Drift on japanese touge circuits while scary p...,[Racing],"[Racing, Driving, Automobile Sim, Arcade, 3D, ...","[Single-player, Steam is learning about this g...","(None, None)",[Racing Games],"(None, None)","(None, None, None)"
4,1786320,The Wolf Is Dead,Late 2022,The Wolf is Dead is a murder mystery game. Whe...,"[Adventure, Indie]","[Detective, Mystery, Puzzle, Crime, Story Rich...","[Single-player, Steam is learning about this g...","(None, None)",[Indie Games],"(None, None)","(None, None, None)"


In [13]:
entries = 0
for thread in threads:
    print('on entry #' + str(entries))
    for entry in thread.game_entries:
            df.loc[entries] = entry
            entries += 1

df.to_csv('df_full.csv')
df.head(5)

on entry #0


C:\Users\ethan\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


on entry #8284
on entry #16568
on entry #24852
on entry #33136
on entry #41420
on entry #49704
on entry #57988


,app_id,game_name,release_date,game_blurb,developer_genres_list,user_tags_list,game_features_list,maturity_rating_data,genre_and_franchise_data,critic_data,reviews_data
0,730,Counter-Strike: Global Offensive,"Aug 21, 2012",Counter-Strike: Global Offensive (CS: GO) expa...,"[Action, Free to Play]","[FPS, Shooter, Multiplayer, Competitive, Actio...","[Steam Achievements, Full controller support, ...","(None, None)",[Free to Play Games],"(83, metacritic)","(Very Positive, Very Positive, 6731720)"
1,1938090,Call of Duty®: Modern Warfare® II,"Oct 27, 2022",Call of Duty®: Modern Warfare® II drops player...,[Action],"[FPS, Action, Shooter, Multiplayer, Military, ...","[Single-player, Online PvP, Online Co-op, Cros...","(M, ESRB)","[Action Games, Call of Duty Franchise]","(None, None)","(None, None, None)"
2,236390,War Thunder,"Aug 15, 2013",War Thunder is the most comprehensive free-to-...,"[Action, Free to Play, Massively Multiplayer, ...","[Free to Play, Vehicular Combat, Combat, VR, W...","[Single-player, MMO, Online PvP, Online Co-op,...","(T, ESRB)",[Massively Multiplayer Games],"(81, metacritic)","(Mostly Positive, Mostly Positive, 341555)"
3,1172470,Apex Legends™,"Nov 4, 2020","Apex Legends is the award-winning, free-to-pla...","[Action, Adventure, Free to Play]","[Free to Play, Multiplayer, Battle Royale, Sho...","[Online PvP, Online Co-op, Steam Achievements,...","(T, ESRB)","[Action Games, Apex Legends Official Franchise]","(88, metacritic)","(Very Positive, Mostly Positive, 527702)"
4,1599340,Lost Ark,"Feb 11, 2022",Embark on an odyssey for the Lost Ark in a vas...,"[Action, Adventure, Free to Play, Massively Mu...","[MMORPG, Free to Play, Action RPG, RPG, Multip...","[Single-player, MMO, Online PvP, Online Co-op,...","(M, ESRB)",[Free to Play Games],"(None, None)","(Mostly Positive, Mostly Positive, 174373)"


# Data Cleaning

Describe your data cleaning steps here.

In [3]:
list_converter = lambda x: x.strip("[]").strip("()").lower().split(", ") # need to read lists inside the csv!
df = pd.read_csv(
    'df_full.csv', 
    converters={
        "developer_genres_list": list_converter, 
        'user_tags_list': list_converter,
        'game_features_list': list_converter,
        'maturity_rating_data': list_converter,
        'genre_and_franchise_data': list_converter,
        'critic_data': list_converter,
        'reviews_data': list_converter
    })


In [4]:
print(df.shape)
len(pd.unique(df['app_id']))

# use a copy to avoid messing things up !
# the threading above is not the cleanest, always de-dupe just to be sure
df_clean = df.drop_duplicates(subset='app_id', keep="first")
df_clean.head(5)

(66272, 12)


,Unnamed: 0,app_id,game_name,release_date,game_blurb,developer_genres_list,user_tags_list,game_features_list,maturity_rating_data,genre_and_franchise_data,critic_data,reviews_data
0,0,730,Counter-Strike: Global Offensive,"Aug 21, 2012",Counter-Strike: Global Offensive (CS: GO) expa...,"['action', 'free to play']","['fps', 'shooter', 'multiplayer', 'competitive...","['steam achievements', 'full controller suppor...","[none, none]",['free to play games'],"['83', 'metacritic']","['very positive', 'very positive', '6731720']"
1,1,1938090,Call of Duty®: Modern Warfare® II,"Oct 27, 2022",Call of Duty®: Modern Warfare® II drops player...,['action'],"['fps', 'action', 'shooter', 'multiplayer', 'm...","['single-player', 'online pvp', 'online co-op'...","['m', 'esrb']","['action games', 'call of duty franchise']","[none, none]","[none, none, none]"
2,2,236390,War Thunder,"Aug 15, 2013",War Thunder is the most comprehensive free-to-...,"['action', 'free to play', 'massively multipla...","['free to play', 'vehicular combat', 'combat',...","['single-player', 'mmo', 'online pvp', 'online...","['t', 'esrb']",['massively multiplayer games'],"['81', 'metacritic']","['mostly positive', 'mostly positive', '341555']"
3,3,1172470,Apex Legends™,"Nov 4, 2020","Apex Legends is the award-winning, free-to-pla...","['action', 'adventure', 'free to play']","['free to play', 'multiplayer', 'battle royale...","['online pvp', 'online co-op', 'steam achievem...","['t', 'esrb']","['action games', 'apex legends official franch...","['88', 'metacritic']","['very positive', 'mostly positive', '527702']"
4,4,1599340,Lost Ark,"Feb 11, 2022",Embark on an odyssey for the Lost Ark in a vas...,"['action', 'adventure', 'free to play', 'massi...","['mmorpg', 'free to play', 'action rpg', 'rpg'...","['single-player', 'mmo', 'online pvp', 'online...","['m', 'esrb']",['free to play games'],"[none, none]","['mostly positive', 'mostly positive', '174373']"


In [6]:
# during row generation, some tuples were used to reduce computational complexity
# however, for non-list entries, this data should be extracted into single-use columns
# whenever possible
df_clean['critic_score'] = [item[0].strip('\'') for item in df_clean['critic_data']]
df_clean['critic_name'] = [item[1].strip('\'') for item in df_clean['critic_data']]
df_clean['total_review_score'] = [item[0].strip('\'') for item in df_clean['reviews_data']]
df_clean['recent_review_score'] = [item[1].strip('\'') for item in df_clean['reviews_data']]
df_clean['review_count'] = [item[2].strip('\'') for item in df_clean['reviews_data']]
# df_clean['genre'] = [item[0] for item in df_clean['genre_and_franchise_data']] # may want to clean up later
df_clean['maturity_rating_score'] = [item[0].strip('\'') for item in df_clean['maturity_rating_data']]
df_clean['maturity_rating_agency'] = [item[1].strip('\'') for item in df_clean['maturity_rating_data']]

df_clean.head(5)

C:\Users\ethan\AppData\Local\Temp\ipykernel_18080\2339414307.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['critic_score'] = [item[0].strip('\'') for item in df_clean['critic_data']]
C:\Users\ethan\AppData\Local\Temp\ipykernel_18080\2339414307.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['critic_name'] = [item[1].strip('\'') for item in df_clean['critic_data']]
C:\Users\ethan\AppData\Local\Temp\ipykernel_18080\2339414307.py:6: SettingWithCopyWarning: 
A value is trying to be s

,Unnamed: 0,app_id,game_name,release_date,game_blurb,developer_genres_list,user_tags_list,game_features_list,maturity_rating_data,genre_and_franchise_data,critic_data,reviews_data,critic_score,critic_name,total_review_score,recent_review_score,review_count,maturity_rating_score,maturity_rating_agency
0,0,730,Counter-Strike: Global Offensive,"Aug 21, 2012",Counter-Strike: Global Offensive (CS: GO) expa...,"['action', 'free to play']","['fps', 'shooter', 'multiplayer', 'competitive...","['steam achievements', 'full controller suppor...","[none, none]",['free to play games'],"['83', 'metacritic']","['very positive', 'very positive', '6731720']",83,metacritic,very positive,very positive,6731720,none,none
1,1,1938090,Call of Duty®: Modern Warfare® II,"Oct 27, 2022",Call of Duty®: Modern Warfare® II drops player...,['action'],"['fps', 'action', 'shooter', 'multiplayer', 'm...","['single-player', 'online pvp', 'online co-op'...","['m', 'esrb']","['action games', 'call of duty franchise']","[none, none]","[none, none, none]",none,none,none,none,none,m,esrb
2,2,236390,War Thunder,"Aug 15, 2013",War Thunder is the most comprehensive free-to-...,"['action', 'free to play', 'massively multipla...","['free to play', 'vehicular combat', 'combat',...","['single-player', 'mmo', 'online pvp', 'online...","['t', 'esrb']",['massively multiplayer games'],"['81', 'metacritic']","['mostly positive', 'mostly positive', '341555']",81,metacritic,mostly positive,mostly positive,341555,t,esrb
3,3,1172470,Apex Legends™,"Nov 4, 2020","Apex Legends is the award-winning, free-to-pla...","['action', 'adventure', 'free to play']","['free to play', 'multiplayer', 'battle royale...","['online pvp', 'online co-op', 'steam achievem...","['t', 'esrb']","['action games', 'apex legends official franch...","['88', 'metacritic']","['very positive', 'mostly positive', '527702']",88,metacritic,very positive,mostly positive,527702,t,esrb
4,4,1599340,Lost Ark,"Feb 11, 2022",Embark on an odyssey for the Lost Ark in a vas...,"['action', 'adventure', 'free to play', 'massi...","['mmorpg', 'free to play', 'action rpg', 'rpg'...","['single-player', 'mmo', 'online pvp', 'online...","['m', 'esrb']",['free to play games'],"[none, none]","['mostly positive', 'mostly positive', '174373']",none,none,mostly positive,mostly positive,174373,m,esrb


In [7]:
# make sure our dates are in a useful format
# df_clean['release_date'] = pd.to_datetime(df_clean['release_date']) breaks on some inputs, but unused right now

# lowercase all of the text data
df_clean.loc[:, 'game_name'] = df_clean['game_name'].str.lower()
df_clean.loc[:, 'game_blurb'] = df_clean['game_blurb'].str.lower()

C:\Users\ethan\AppData\Local\Temp\ipykernel_18080\3105876860.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.loc[:, 'game_name'] = df_clean['game_name'].str.lower()
C:\Users\ethan\AppData\Local\Temp\ipykernel_18080\3105876860.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.loc[:, 'game_blurb'] = df_clean['game_blurb'].str.lower()


In [8]:
df_clean.head(5)

,Unnamed: 0,app_id,game_name,release_date,game_blurb,developer_genres_list,user_tags_list,game_features_list,maturity_rating_data,genre_and_franchise_data,critic_data,reviews_data,critic_score,critic_name,total_review_score,recent_review_score,review_count,maturity_rating_score,maturity_rating_agency
0,0,730,counter-strike: global offensive,"Aug 21, 2012",counter-strike: global offensive (cs: go) expa...,"['action', 'free to play']","['fps', 'shooter', 'multiplayer', 'competitive...","['steam achievements', 'full controller suppor...","[none, none]",['free to play games'],"['83', 'metacritic']","['very positive', 'very positive', '6731720']",83,metacritic,very positive,very positive,6731720,none,none
1,1,1938090,call of duty®: modern warfare® ii,"Oct 27, 2022",call of duty®: modern warfare® ii drops player...,['action'],"['fps', 'action', 'shooter', 'multiplayer', 'm...","['single-player', 'online pvp', 'online co-op'...","['m', 'esrb']","['action games', 'call of duty franchise']","[none, none]","[none, none, none]",none,none,none,none,none,m,esrb
2,2,236390,war thunder,"Aug 15, 2013",war thunder is the most comprehensive free-to-...,"['action', 'free to play', 'massively multipla...","['free to play', 'vehicular combat', 'combat',...","['single-player', 'mmo', 'online pvp', 'online...","['t', 'esrb']",['massively multiplayer games'],"['81', 'metacritic']","['mostly positive', 'mostly positive', '341555']",81,metacritic,mostly positive,mostly positive,341555,t,esrb
3,3,1172470,apex legends™,"Nov 4, 2020","apex legends is the award-winning, free-to-pla...","['action', 'adventure', 'free to play']","['free to play', 'multiplayer', 'battle royale...","['online pvp', 'online co-op', 'steam achievem...","['t', 'esrb']","['action games', 'apex legends official franch...","['88', 'metacritic']","['very positive', 'mostly positive', '527702']",88,metacritic,very positive,mostly positive,527702,t,esrb
4,4,1599340,lost ark,"Feb 11, 2022",embark on an odyssey for the lost ark in a vas...,"['action', 'adventure', 'free to play', 'massi...","['mmorpg', 'free to play', 'action rpg', 'rpg'...","['single-player', 'mmo', 'online pvp', 'online...","['m', 'esrb']",['free to play games'],"[none, none]","['mostly positive', 'mostly positive', '174373']",none,none,mostly positive,mostly positive,174373,m,esrb


In [9]:
# steam uses text descriptors to show ratings, this maps them to numbers
# it is uncertain if these values are equidistant from each other, but
# for our purposes, it is close enough. negative values used to make it more intuitive
user_score_map = {
    'overwhelmingly positive': 4,
    'very positive': 3,
    'positive': 2,
    'mostly positive': 1,
    'mixed': 0,
    'mostly negative': -1,
    'negative': -2,
    'very negative': -3,
    'overwhelmingly negative': -4,
}

df_clean['total_review_score_numeric'] = df_clean['total_review_score'].map(user_score_map)
df_clean['recent_review_score_numeric'] = df_clean['recent_review_score'].map(user_score_map)

C:\Users\ethan\AppData\Local\Temp\ipykernel_18080\3032060414.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['total_review_score_numeric'] = df_clean['total_review_score'].map(user_score_map)
C:\Users\ethan\AppData\Local\Temp\ipykernel_18080\3032060414.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['recent_review_score_numeric'] = df_clean['recent_review_score'].map(user_score_map)


In [10]:
df_clean.head(5)

,Unnamed: 0,app_id,game_name,release_date,game_blurb,developer_genres_list,user_tags_list,game_features_list,maturity_rating_data,genre_and_franchise_data,...,reviews_data,critic_score,critic_name,total_review_score,recent_review_score,review_count,maturity_rating_score,maturity_rating_agency,total_review_score_numeric,recent_review_score_numeric
0,0,730,counter-strike: global offensive,"Aug 21, 2012",counter-strike: global offensive (cs: go) expa...,"['action', 'free to play']","['fps', 'shooter', 'multiplayer', 'competitive...","['steam achievements', 'full controller suppor...","[none, none]",['free to play games'],...,"['very positive', 'very positive', '6731720']",83,metacritic,very positive,very positive,6731720,none,none,3.0,3.0
1,1,1938090,call of duty®: modern warfare® ii,"Oct 27, 2022",call of duty®: modern warfare® ii drops player...,['action'],"['fps', 'action', 'shooter', 'multiplayer', 'm...","['single-player', 'online pvp', 'online co-op'...","['m', 'esrb']","['action games', 'call of duty franchise']",...,"[none, none, none]",none,none,none,none,none,m,esrb,NaN,NaN
2,2,236390,war thunder,"Aug 15, 2013",war thunder is the most comprehensive free-to-...,"['action', 'free to play', 'massively multipla...","['free to play', 'vehicular combat', 'combat',...","['single-player', 'mmo', 'online pvp', 'online...","['t', 'esrb']",['massively multiplayer games'],...,"['mostly positive', 'mostly positive', '341555']",81,metacritic,mostly positive,mostly positive,341555,t,esrb,1.0,1.0
3,3,1172470,apex legends™,"Nov 4, 2020","apex legends is the award-winning, free-to-pla...","['action', 'adventure', 'free to play']","['free to play', 'multiplayer', 'battle royale...","['online pvp', 'online co-op', 'steam achievem...","['t', 'esrb']","['action games', 'apex legends official franch...",...,"['very positive', 'mostly positive', '527702']",88,metacritic,very positive,mostly positive,527702,t,esrb,3.0,1.0
4,4,1599340,lost ark,"Feb 11, 2022",embark on an odyssey for the lost ark in a vas...,"['action', 'adventure', 'free to play', 'massi...","['mmorpg', 'free to play', 'action rpg', 'rpg'...","['single-player', 'mmo', 'online pvp', 'online...","['m', 'esrb']",['free to play games'],...,"['mostly positive', 'mostly positive', '174373']",none,none,mostly positive,mostly positive,174373,m,esrb,1.0,1.0


In [11]:
# esrb ratings are letters/phrases, so we map them to numbers to make computation easier
esrb_rating_map = {
    'e': 1, # everyone
    'e10plus': 2, # e10+ (older kids)
    't': 3, # teen
    'm': 4, # mature
    'ao': 5, # adults only
    'rp': 0, # rating pending
}

df_clean['esrb_rating_numeric'] = df_clean[
        df_clean['maturity_rating_agency'] == 'esrb'
    ]['maturity_rating_score'].map(esrb_rating_map)

C:\Users\ethan\AppData\Local\Temp\ipykernel_18080\4246702504.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['esrb_rating_numeric'] = df_clean[


In [12]:
df_clean.head(5)

,Unnamed: 0,app_id,game_name,release_date,game_blurb,developer_genres_list,user_tags_list,game_features_list,maturity_rating_data,genre_and_franchise_data,...,critic_score,critic_name,total_review_score,recent_review_score,review_count,maturity_rating_score,maturity_rating_agency,total_review_score_numeric,recent_review_score_numeric,esrb_rating_numeric
0,0,730,counter-strike: global offensive,"Aug 21, 2012",counter-strike: global offensive (cs: go) expa...,"['action', 'free to play']","['fps', 'shooter', 'multiplayer', 'competitive...","['steam achievements', 'full controller suppor...","[none, none]",['free to play games'],...,83,metacritic,very positive,very positive,6731720,none,none,3.0,3.0,NaN
1,1,1938090,call of duty®: modern warfare® ii,"Oct 27, 2022",call of duty®: modern warfare® ii drops player...,['action'],"['fps', 'action', 'shooter', 'multiplayer', 'm...","['single-player', 'online pvp', 'online co-op'...","['m', 'esrb']","['action games', 'call of duty franchise']",...,none,none,none,none,none,m,esrb,NaN,NaN,4.0
2,2,236390,war thunder,"Aug 15, 2013",war thunder is the most comprehensive free-to-...,"['action', 'free to play', 'massively multipla...","['free to play', 'vehicular combat', 'combat',...","['single-player', 'mmo', 'online pvp', 'online...","['t', 'esrb']",['massively multiplayer games'],...,81,metacritic,mostly positive,mostly positive,341555,t,esrb,1.0,1.0,3.0
3,3,1172470,apex legends™,"Nov 4, 2020","apex legends is the award-winning, free-to-pla...","['action', 'adventure', 'free to play']","['free to play', 'multiplayer', 'battle royale...","['online pvp', 'online co-op', 'steam achievem...","['t', 'esrb']","['action games', 'apex legends official franch...",...,88,metacritic,very positive,mostly positive,527702,t,esrb,3.0,1.0,3.0
4,4,1599340,lost ark,"Feb 11, 2022",embark on an odyssey for the lost ark in a vas...,"['action', 'adventure', 'free to play', 'massi...","['mmorpg', 'free to play', 'action rpg', 'rpg'...","['single-player', 'mmo', 'online pvp', 'online...","['m', 'esrb']",['free to play games'],...,none,none,mostly positive,mostly positive,174373,m,esrb,1.0,1.0,4.0


In [13]:
# re-order/drop some of the columns for legibility
df_clean = df_clean[[
    'app_id',
    'game_name',
    'release_date',
#     'genre',
    'game_blurb',
    'maturity_rating_score',
    'maturity_rating_agency',
    'esrb_rating_numeric',
    'total_review_score_numeric',
    'recent_review_score_numeric',
    'review_count',
    'critic_score',
    'critic_name',
    'user_tags_list',
    'developer_genres_list',
    'game_features_list'
]]

# IMPORTANT NOTE:
# I have not removed None/NaN data from this dataframe! This is because I am uncertain which exact metrics will
# be most important to our analysis. I also have only lightly processed the text, it should have non alphanumeric
# character removed and perhaps be tokenized/stemmed in the EDA step. This document is primarily focused on gathering
# the data from store.steampowered.com so we can manipulate it without having to gather it often.

# write to a file for convenience
df_clean.to_csv('df_clean.csv')
df_clean.head(10) # always sanity check!

,app_id,game_name,release_date,game_blurb,maturity_rating_score,maturity_rating_agency,esrb_rating_numeric,total_review_score_numeric,recent_review_score_numeric,review_count,critic_score,critic_name,user_tags_list,developer_genres_list,game_features_list
0,730,counter-strike: global offensive,"Aug 21, 2012",counter-strike: global offensive (cs: go) expa...,none,none,NaN,3.0,3.0,6731720,83,metacritic,"['fps', 'shooter', 'multiplayer', 'competitive...","['action', 'free to play']","['steam achievements', 'full controller suppor..."
1,1938090,call of duty®: modern warfare® ii,"Oct 27, 2022",call of duty®: modern warfare® ii drops player...,m,esrb,4.0,NaN,NaN,none,none,none,"['fps', 'action', 'shooter', 'multiplayer', 'm...",['action'],"['single-player', 'online pvp', 'online co-op'..."
2,236390,war thunder,"Aug 15, 2013",war thunder is the most comprehensive free-to-...,t,esrb,3.0,1.0,1.0,341555,81,metacritic,"['free to play', 'vehicular combat', 'combat',...","['action', 'free to play', 'massively multipla...","['single-player', 'mmo', 'online pvp', 'online..."
3,1172470,apex legends™,"Nov 4, 2020","apex legends is the award-winning, free-to-pla...",t,esrb,3.0,3.0,1.0,527702,88,metacritic,"['free to play', 'multiplayer', 'battle royale...","['action', 'adventure', 'free to play']","['online pvp', 'online co-op', 'steam achievem..."
4,1599340,lost ark,"Feb 11, 2022",embark on an odyssey for the lost ark in a vas...,m,esrb,4.0,1.0,1.0,174373,none,none,"['mmorpg', 'free to play', 'action rpg', 'rpg'...","['action', 'adventure', 'free to play', 'massi...","['single-player', 'mmo', 'online pvp', 'online..."
5,294100,rimworld,"Oct 17, 2018",a sci-fi colony sim driven by an intelligent a...,none,none,NaN,4.0,4.0,121410,87,metacritic,"['colony sim', 'base building', 'survival', 's...","['indie', 'simulation', 'strategy']","['single-player', 'steam workshop', 'steam clo..."
6,1085660,destiny 2,"Oct 1, 2019",destiny 2 is an action mmo with a single evolv...,t,esrb,3.0,3.0,1.0,501974,83,metacritic,"['free to play', 'open world', 'fps', 'looter ...","['action', 'adventure', 'free to play']","['single-player', 'online pvp', 'online co-op'..."
7,1063730,new world,"Sep 28, 2021","explore a thrilling, open-world mmo filled wit...",t,esrb,3.0,0.0,3.0,204606,70,metacritic,"['massively multiplayer', 'open world', 'mmorp...","['action', 'adventure', 'massively multiplayer...","['mmo', 'online pvp', 'online co-op', 'steam a..."
8,1687950,persona 5 royal,"Oct 20, 2022",don the mask and join the phantom thieves of h...,m,esrb,4.0,NaN,NaN,none,95,metacritic,"['jrpg', 'anime', 'story rich', 'rpg', 'party-...",['rpg'],"['single-player', 'steam achievements', 'full ..."
9,548430,deep rock galactic,"May 13, 2020",deep rock galactic is a 1-4 player co-op fps f...,none,none,NaN,4.0,4.0,122870,85,metacritic,"['co-op', 'pve', 'fps', 'exploration', 'loot',...",['action'],"['single-player', 'online co-op', 'steam achie..."
